In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv
/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv
/kaggle/input/equity-post-HCT-survival-predictions/train.csv
/kaggle/input/equity-post-HCT-survival-predictions/test.csv


# Importing useful packages & Loading Data

In [57]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import missingno as msno
import statistics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [58]:
train = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
#train = train.dropna()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Data columns (total 60 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      28800 non-null  int64  
 1   dri_score               28646 non-null  object 
 2   psych_disturb           26738 non-null  object 
 3   cyto_score              20732 non-null  object 
 4   diabetes                26681 non-null  object 
 5   hla_match_c_high        24180 non-null  float64
 6   hla_high_res_8          22971 non-null  float64
 7   tbi_status              28800 non-null  object 
 8   arrhythmia              26598 non-null  object 
 9   hla_low_res_6           25530 non-null  float64
 10  graft_type              28800 non-null  object 
 11  vent_hist               28541 non-null  object 
 12  renal_issue             26885 non-null  object 
 13  pulm_severe             26665 non-null  object 
 14  prim_disease_hct        28800 non-null

In [59]:
train['efs'].head()

0    0.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: efs, dtype: float64

# Factorizing Categorical Variables

In [60]:
categorical_columns = train.select_dtypes(include='object').columns
print(categorical_columns)

Index(['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status',
       'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe',
       'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab',
       'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity',
       'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe',
       'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match',
       'race_group', 'hepatic_mild', 'tce_div_match', 'donor_related',
       'melphalan_dose', 'cardiac', 'pulm_moderate'],
      dtype='object')


In [61]:
train['dri_score_numeric'] = pd.factorize(train['dri_score'])[0]
train['psych_disturb_numeric'] = pd.factorize(train['psych_disturb'])[0]
train['cyto_score_numeric'] = pd.factorize(train['cyto_score'])[0]
train['diabetes_numeric'] = pd.factorize(train['diabetes'])[0]
train['tbi_status_numeric'] = pd.factorize(train['tbi_status'])[0]
train['arrhythmia_numeric'] = pd.factorize(train['arrhythmia'])[0]
train['graft_type_numeric'] = pd.factorize(train['graft_type'])[0]
train['vent_hist_numeric'] = pd.factorize(train['vent_hist'])[0]
train['renal_issue_numeric'] = pd.factorize(train['renal_issue'])[0]
train['pulm_severe_numeric'] = pd.factorize(train['pulm_severe'])[0]
train['prim_disease_hct_numeric'] = pd.factorize(train['prim_disease_hct'])[0]
train['cmv_status_numeric'] = pd.factorize(train['cmv_status'])[0]
train['tce_imm_match_numeric'] = pd.factorize(train['tce_imm_match'])[0]
train['rituximab_numeric'] = pd.factorize(train['rituximab'])[0]
train['prod_type_numeric'] = pd.factorize(train['cyto_score_detail'])[0]
train['conditioning_intensity_numeric'] = pd.factorize(train['conditioning_intensity'])[0]
train['ethnicity_numeric'] = pd.factorize(train['ethnicity'])[0]
train['obesity_numeric'] = pd.factorize(train['obesity'])[0]
train['mrd_hct_numeric'] = pd.factorize(train['mrd_hct'])[0]
train['in_vivo_tcd_numeric'] = pd.factorize(train['in_vivo_tcd'])[0]
train['tce_match_numeric'] = pd.factorize(train['tce_match'])[0]
train['hepatic_severe_numeric'] = pd.factorize(train['hepatic_severe'])[0]
train['prior_tumor_numeric'] = pd.factorize(train['prior_tumor'])[0]
train['peptic_ulcer_numeric'] = pd.factorize(train['peptic_ulcer'])[0]
train['gvhd_proph_numeric'] = pd.factorize(train['gvhd_proph'])[0]
train['rheum_issue_numeric'] = pd.factorize(train['rheum_issue'])[0]
train['sex_match_numeric'] = pd.factorize(train['sex_match'])[0]
train['race_group_numeric'] = pd.factorize(train['race_group'])[0]
train['hepatic_mild_numeric'] = pd.factorize(train['hepatic_mild'])[0]
train['tce_div_match_numeric'] = pd.factorize(train['tce_div_match'])[0]
train['donor_related_numeric'] = pd.factorize(train['donor_related'])[0]
train['melphalan_dose_numeric'] = pd.factorize(train['melphalan_dose'])[0]
train['cardiac_numeric'] = pd.factorize(train['cardiac'])[0]
train['pulm_moderate_numeric'] = pd.factorize(train['pulm_moderate'])[0]

# Preparing Data for Modeling

In [62]:
train.select_dtypes(include=['int64','float64']).columns

Index(['ID', 'hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6',
       'hla_high_res_6', 'hla_high_res_10', 'hla_match_dqb1_high',
       'hla_nmdp_6', 'hla_match_c_low', 'hla_match_drb1_low',
       'hla_match_dqb1_low', 'year_hct', 'hla_match_a_high', 'donor_age',
       'hla_match_b_low', 'age_at_hct', 'hla_match_a_low', 'hla_match_b_high',
       'comorbidity_score', 'karnofsky_score', 'hla_low_res_8',
       'hla_match_drb1_high', 'hla_low_res_10', 'efs', 'efs_time',
       'dri_score_numeric', 'psych_disturb_numeric', 'cyto_score_numeric',
       'diabetes_numeric', 'tbi_status_numeric', 'arrhythmia_numeric',
       'graft_type_numeric', 'vent_hist_numeric', 'renal_issue_numeric',
       'pulm_severe_numeric', 'prim_disease_hct_numeric', 'cmv_status_numeric',
       'tce_imm_match_numeric', 'rituximab_numeric', 'prod_type_numeric',
       'conditioning_intensity_numeric', 'ethnicity_numeric',
       'obesity_numeric', 'mrd_hct_numeric', 'in_vivo_tcd_numeric',
       'tce_matc

In [63]:
y = train.efs
y2 = train.efs_time
features = ['hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6',
       'hla_high_res_6', 'hla_high_res_10', 'hla_match_dqb1_high',
       'hla_nmdp_6', 'hla_match_c_low', 'hla_match_drb1_low',
       'hla_match_dqb1_low', 'year_hct', 'hla_match_a_high', 'donor_age',
       'hla_match_b_low', 'age_at_hct', 'hla_match_a_low', 'hla_match_b_high',
       'comorbidity_score', 'karnofsky_score', 'hla_low_res_8',
       'hla_match_drb1_high', 'hla_low_res_10', 'efs_time',
       'dri_score_numeric', 'psych_disturb_numeric', 'cyto_score_numeric',
       'diabetes_numeric', 'tbi_status_numeric', 'arrhythmia_numeric',
       'graft_type_numeric', 'vent_hist_numeric', 'renal_issue_numeric',
       'pulm_severe_numeric', 'prim_disease_hct_numeric', 'cmv_status_numeric',
       'tce_imm_match_numeric', 'rituximab_numeric', 'prod_type_numeric',
       'conditioning_intensity_numeric', 'ethnicity_numeric',
       'obesity_numeric', 'mrd_hct_numeric', 'in_vivo_tcd_numeric',
       'tce_match_numeric', 'hepatic_severe_numeric', 'prior_tumor_numeric',
       'peptic_ulcer_numeric', 'gvhd_proph_numeric', 'rheum_issue_numeric',
       'sex_match_numeric', 'race_group_numeric', 'hepatic_mild_numeric',
       'tce_div_match_numeric', 'donor_related_numeric',
       'melphalan_dose_numeric', 'cardiac_numeric', 'pulm_moderate_numeric']

features2 = ['ID', 'hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6',
       'hla_high_res_6', 'hla_high_res_10', 'hla_match_dqb1_high',
       'hla_nmdp_6', 'hla_match_c_low', 'hla_match_drb1_low',
       'hla_match_dqb1_low', 'year_hct', 'hla_match_a_high', 'donor_age',
       'hla_match_b_low', 'age_at_hct', 'hla_match_a_low', 'hla_match_b_high',
       'comorbidity_score', 'karnofsky_score', 'hla_low_res_8',
       'hla_match_drb1_high', 'hla_low_res_10',
       'dri_score_numeric', 'psych_disturb_numeric', 'cyto_score_numeric',
       'diabetes_numeric', 'tbi_status_numeric', 'arrhythmia_numeric',
       'graft_type_numeric', 'vent_hist_numeric', 'renal_issue_numeric',
       'pulm_severe_numeric', 'prim_disease_hct_numeric', 'cmv_status_numeric',
       'tce_imm_match_numeric', 'rituximab_numeric', 'prod_type_numeric',
       'conditioning_intensity_numeric', 'ethnicity_numeric',
       'obesity_numeric', 'mrd_hct_numeric', 'in_vivo_tcd_numeric',
       'tce_match_numeric', 'hepatic_severe_numeric', 'prior_tumor_numeric',
       'peptic_ulcer_numeric', 'gvhd_proph_numeric', 'rheum_issue_numeric',
       'sex_match_numeric', 'race_group_numeric', 'hepatic_mild_numeric',
       'tce_div_match_numeric', 'donor_related_numeric',
       'melphalan_dose_numeric', 'cardiac_numeric', 'pulm_moderate_numeric']
X = train[features]
X2 = train[features2]

In [64]:
X = X.fillna(value = 0)
X2 = X2.fillna(value = 0)

In [65]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Data columns (total 57 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hla_match_c_high                28800 non-null  float64
 1   hla_high_res_8                  28800 non-null  float64
 2   hla_low_res_6                   28800 non-null  float64
 3   hla_high_res_6                  28800 non-null  float64
 4   hla_high_res_10                 28800 non-null  float64
 5   hla_match_dqb1_high             28800 non-null  float64
 6   hla_nmdp_6                      28800 non-null  float64
 7   hla_match_c_low                 28800 non-null  float64
 8   hla_match_drb1_low              28800 non-null  float64
 9   hla_match_dqb1_low              28800 non-null  float64
 10  year_hct                        28800 non-null  int64  
 11  hla_match_a_high                28800 non-null  float64
 12  donor_age                       

In [66]:
#looking for best fit for efs(target Variable)
feat_select = SelectKBest(f_classif, k='all')
feat_select.fit_transform(X, y)
feat_pvals = pd.DataFrame({'Feature' : X.columns, 'p_value' : feat_select.pvalues_}).sort_values('p_value') 
feat_pvals[feat_pvals['p_value'] < 0.05]

,Feature,p_value
29,graft_type_numeric,0.000000e+00
22,efs_time,0.000000e+00
14,age_at_hct,0.000000e+00
38,conditioning_intensity_numeric,1.337265e-295
37,prod_type_numeric,6.003531e-218
42,in_vivo_tcd_numeric,5.339235e-166
17,comorbidity_score,5.605531e-132
25,cyto_score_numeric,9.670950e-122
10,year_hct,2.628877e-73
12,donor_age,2.645166e-65


In [67]:
#Looking for best fit for efs_time(Secondary target variable)
feat_select2 = SelectKBest(f_classif, k='all')
feat_select2.fit_transform(X2, y2)
feat_pvals2 = pd.DataFrame({'Feature' : X2.columns, 'p_value' : feat_select2.pvalues_}).sort_values('p_value') 
feat_pvals2[feat_pvals2['p_value'] < 0.05]

,Feature,p_value
29,graft_type_numeric,5.817417e-55
15,age_at_hct,7.652135e-44
38,conditioning_intensity_numeric,7.565546e-43
19,karnofsky_score,3.712416e-25
42,in_vivo_tcd_numeric,1.212225e-19
44,hepatic_severe_numeric,1.674005e-19
11,year_hct,1.878153e-16
23,dri_score_numeric,2.451736e-16
13,donor_age,5.359088e-13
47,gvhd_proph_numeric,1.188034e-11


In [68]:
feat_pvals['Feature'].values

array(['graft_type_numeric', 'efs_time', 'age_at_hct',
       'conditioning_intensity_numeric', 'prod_type_numeric',
       'in_vivo_tcd_numeric', 'comorbidity_score', 'cyto_score_numeric',
       'year_hct', 'donor_age', 'dri_score_numeric', 'mrd_hct_numeric',
       'prior_tumor_numeric', 'hepatic_severe_numeric',
       'pulm_severe_numeric', 'psych_disturb_numeric',
       'hla_match_drb1_high', 'pulm_moderate_numeric', 'cardiac_numeric',
       'hla_match_drb1_low', 'diabetes_numeric', 'ethnicity_numeric',
       'cmv_status_numeric', 'donor_related_numeric', 'hla_low_res_6',
       'hla_match_b_low', 'sex_match_numeric', 'race_group_numeric',
       'gvhd_proph_numeric', 'hla_nmdp_6', 'tbi_status_numeric',
       'obesity_numeric', 'arrhythmia_numeric', 'hla_low_res_8',
       'karnofsky_score', 'renal_issue_numeric', 'hla_match_dqb1_low',
       'peptic_ulcer_numeric', 'hla_high_res_6', 'hla_high_res_8',
       'hla_match_dqb1_high', 'prim_disease_hct_numeric',
       'hla_match

In [69]:
feat_pvals2['Feature'].values

array(['graft_type_numeric', 'age_at_hct',
       'conditioning_intensity_numeric', 'karnofsky_score',
       'in_vivo_tcd_numeric', 'hepatic_severe_numeric', 'year_hct',
       'dri_score_numeric', 'donor_age', 'gvhd_proph_numeric',
       'tce_div_match_numeric', 'cmv_status_numeric', 'ethnicity_numeric',
       'tce_imm_match_numeric', 'cyto_score_numeric',
       'peptic_ulcer_numeric', 'hla_match_drb1_high', 'hla_high_res_8',
       'vent_hist_numeric', 'hla_nmdp_6', 'rheum_issue_numeric',
       'hla_high_res_6', 'race_group_numeric', 'mrd_hct_numeric',
       'hla_high_res_10', 'hla_match_a_high', 'hla_match_b_low',
       'hla_match_drb1_low', 'hla_low_res_8', 'hla_low_res_6',
       'hla_match_b_high', 'hla_low_res_10', 'hla_match_c_high',
       'hla_match_dqb1_low', 'hla_match_a_low', 'hla_match_c_low',
       'tce_match_numeric', 'hla_match_dqb1_high', 'rituximab_numeric',
       'prim_disease_hct_numeric', 'donor_related_numeric',
       'tbi_status_numeric', 'ID', 'melpha

In [70]:
features = ['graft_type_numeric', 'efs_time', 'age_at_hct']
X = X[features]

  'conditioning_intensity_numeric', 'prod_type_numeric',
       'in_vivo_tcd_numeric', 'comorbidity_score', 'cyto_score_numeric',
       'year_hct', 'donor_age', 'dri_score_numeric', 'mrd_hct_numeric',
       'prior_tumor_numeric', 'hepatic_severe_numeric',
       'pulm_severe_numeric', 'psych_disturb_numeric',
       'hla_match_drb1_high', 'pulm_moderate_numeric', 'cardiac_numeric',
       'hla_match_drb1_low', 'diabetes_numeric', 'ethnicity_numeric',
       'cmv_status_numeric', 'donor_related_numeric', 'hla_low_res_6',
       'hla_match_b_low', 'sex_match_numeric', 'race_group_numeric',
       'gvhd_proph_numeric', 'hla_nmdp_6', 'tbi_status_numeric',
       'obesity_numeric', 'arrhythmia_numeric', 'hla_low_res_8',
       'karnofsky_score', 'renal_issue_numeric', 'hla_match_dqb1_low',
       'peptic_ulcer_numeric', 'hla_high_res_6', 'hla_high_res_8',
       'hla_match_dqb1_high', 'prim_disease_hct_numeric',
       'hla_match_a_low', 'melphalan_dose_numeric',
       'tce_imm_match_numeric', 'vent_hist_numeric', 'tce_match_numeric',
       'tce_div_match_numeric', 'rheum_issue_numeric',
       'hepatic_mild_numeric', 'hla_match_c_high', 'hla_high_res_10',
       'hla_low_res_10', 'hla_match_a_high', 'hla_match_c_low',
       'rituximab_numeric', 'hla_match_b_high']

In [71]:
features2 = ['graft_type_numeric', 'age_at_hct',
       'conditioning_intensity_numeric', 'karnofsky_score',
       'in_vivo_tcd_numeric', 'hepatic_severe_numeric', 'year_hct',
       'dri_score_numeric', 'donor_age', 'gvhd_proph_numeric',
       'tce_div_match_numeric', 'cmv_status_numeric', 'ethnicity_numeric',
       'tce_imm_match_numeric', 'cyto_score_numeric',
       'peptic_ulcer_numeric', 'hla_match_drb1_high', 'hla_high_res_8',
       'vent_hist_numeric', 'hla_nmdp_6', 'rheum_issue_numeric',
       'hla_high_res_6', 'race_group_numeric', 'mrd_hct_numeric',
       'hla_high_res_10', 'hla_match_a_high', 'hla_match_b_low',
       'hla_match_drb1_low', 'hla_low_res_8', 'hla_low_res_6',
       'hla_match_b_high', 'hla_low_res_10', 'hla_match_c_high',
       'hla_match_dqb1_low', 'hla_match_a_low', 'hla_match_c_low',
       'tce_match_numeric', 'hla_match_dqb1_high', 'rituximab_numeric',
       'prim_disease_hct_numeric', 'donor_related_numeric',
       'tbi_status_numeric', 'melphalan_dose_numeric',
       'hepatic_mild_numeric', 'prod_type_numeric', 'sex_match_numeric',
       'renal_issue_numeric', 'prior_tumor_numeric', 'arrhythmia_numeric',
       'diabetes_numeric', 'obesity_numeric', 'cardiac_numeric',
       'pulm_moderate_numeric', 'pulm_severe_numeric',
       'psych_disturb_numeric', 'comorbidity_score']
X2 = X2[features2]

# Creating Models

In [72]:
#Model 1 - efs predict
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=2)

In [73]:
#Model 2 - efs_time predict
train_X2, val_X2, train_y2, val_y2 = train_test_split(X2, y2, random_state=2)

In [74]:
#Model 1 - efs predict
rf_model = RandomForestRegressor(random_state=2)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))
print('The accuracy of the model is: ', rf_model.score(val_X, val_y)) 
print('The accuracy of the training model is: ', rf_model.score(train_X, train_y))

Validation MAE for Random Forest Model: 0
The accuracy of the model is:  0.9260657692804355
The accuracy of the training model is:  0.9875592070432889


In [75]:
#Model 2 - efs_time predict
rf_model2 = RandomForestRegressor(random_state=2)
rf_model2.fit(train_X2, train_y2)
rf_val_predictions2 = rf_model2.predict(val_X2)
rf_val_mae2 = mean_absolute_error(rf_val_predictions2, val_y2)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae2))
print('The accuracy of the model is: ', rf_model2.score(val_X2, val_y2)) 
print('The accuracy of the training model is: ', rf_model2.score(train_X2, train_y2))

Validation MAE for Random Forest Model: 17
The accuracy of the model is:  0.1531075187798312
The accuracy of the training model is:  0.8827780934351473


In [76]:
#Model 3 - Running a linear regression model too on just the efs predict features
linear_model = LinearRegression()
linear_model.fit(train_X, train_y)
linear_val_predictions = rf_model.predict(val_X)
linear_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(linear_val_mae))
print('The accuracy of the model is: ', linear_model.score(val_X, val_y)) 
print('The accuracy of the training model is: ', linear_model.score(train_X, train_y))

Validation MAE for Random Forest Model: 0
The accuracy of the model is:  0.5751820304892818
The accuracy of the training model is:  0.5661308853037306


In [77]:
#Model 1 - efs predict
rf_model_on_full_data = RandomForestRegressor(random_state=2)

# fit rf_model_on_full_data on all data from the training data
rf_model_on_full_data.fit(X,y)

RandomForestRegressor(random_state=2)

In [78]:
#Model 2 - efs_time predict
rf_model_on_full_data2 = RandomForestRegressor(random_state=2)

# fit rf_model_on_full_data on all data from the training data
rf_model_on_full_data2.fit(train_X2,train_y2)

RandomForestRegressor(random_state=2)

In [79]:
test = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

test['dri_score_numeric'] = pd.factorize(test['dri_score'])[0]
test['psych_disturb_numeric'] = pd.factorize(test['psych_disturb'])[0]
test['cyto_score_numeric'] = pd.factorize(test['cyto_score'])[0]
test['diabetes_numeric'] = pd.factorize(test['diabetes'])[0]
test['tbi_status_numeric'] = pd.factorize(test['tbi_status'])[0]
test['arrhythmia_numeric'] = pd.factorize(test['arrhythmia'])[0]
test['graft_type_numeric'] = pd.factorize(test['graft_type'])[0]
test['vent_hist_numeric'] = pd.factorize(test['vent_hist'])[0]
test['renal_issue_numeric'] = pd.factorize(test['renal_issue'])[0]
test['pulm_severe_numeric'] = pd.factorize(test['pulm_severe'])[0]
test['prim_disease_hct_numeric'] = pd.factorize(test['prim_disease_hct'])[0]
test['cmv_status_numeric'] = pd.factorize(test['cmv_status'])[0]
test['tce_imm_match_numeric'] = pd.factorize(test['tce_imm_match'])[0]
test['rituximab_numeric'] = pd.factorize(test['rituximab'])[0]
test['prod_type_numeric'] = pd.factorize(test['cyto_score_detail'])[0]
test['conditioning_intensity_numeric'] = pd.factorize(test['conditioning_intensity'])[0]
test['ethnicity_numeric'] = pd.factorize(test['ethnicity'])[0]
test['obesity_numeric'] = pd.factorize(test['obesity'])[0]
test['mrd_hct_numeric'] = pd.factorize(test['mrd_hct'])[0]
test['in_vivo_tcd_numeric'] = pd.factorize(test['in_vivo_tcd'])[0]
test['tce_match_numeric'] = pd.factorize(test['tce_match'])[0]
test['hepatic_severe_numeric'] = pd.factorize(test['hepatic_severe'])[0]
test['prior_tumor_numeric'] = pd.factorize(test['prior_tumor'])[0]
test['peptic_ulcer_numeric'] = pd.factorize(test['peptic_ulcer'])[0]
test['gvhd_proph_numeric'] = pd.factorize(test['gvhd_proph'])[0]
test['rheum_issue_numeric'] = pd.factorize(test['rheum_issue'])[0]
test['sex_match_numeric'] = pd.factorize(test['sex_match'])[0]
test['race_group_numeric'] = pd.factorize(test['race_group'])[0]
test['hepatic_mild_numeric'] = pd.factorize(test['hepatic_mild'])[0]
test['tce_div_match_numeric'] = pd.factorize(test['tce_div_match'])[0]
test['donor_related_numeric'] = pd.factorize(test['donor_related'])[0]
test['melphalan_dose_numeric'] = pd.factorize(test['melphalan_dose'])[0]
test['cardiac_numeric'] = pd.factorize(test['cardiac'])[0]
test['pulm_moderate_numeric'] = pd.factorize(test['pulm_moderate'])[0]



In [80]:
#Model 2 - efs_time predict
test_X2 = test[features2].fillna(value = 0)

test_preds2 = rf_model_on_full_data2.predict(test_X2)




In [82]:
#Model 3 - Linear Regression
#Model 2 - efs_time predict
linear_model_on_full_data = LinearRegression()

# fit rf_model_on_full_data on all data from the training data
linear_model_on_full_data.fit(X,y)

LinearRegression()

# Final Model and Submission

In [83]:
#Model 1 RFR- efs predict
test['efs_time'] = test_preds2
test_X = test[features].fillna(value = 0)

test_preds = rf_model_on_full_data.predict(test_X)

In [84]:
#Model 3 LR- efs predict
test['efs_time'] = test_preds2
test_X = test[features].fillna(value = 0)

test_preds3 = linear_model_on_full_data.predict(test_X)

In [85]:
output = pd.DataFrame({'ID': test.ID,
                       'prediction': test_preds})
output2 = pd.DataFrame({'ID': test.ID,
                       'prediction': test_preds3})


In [86]:
#RandomForestRegressor Results - Score 0.506
output

,ID,prediction
0,28800,0.00
1,28801,0.96
2,28802,0.11


In [87]:
#Linear Regression Results - Score 0.505
output2

,ID,prediction
0,28800,0.147590
1,28801,0.744337
2,28802,0.584444


In [88]:
output2.to_csv('submission.csv', index=False)